<img src="https://www.earthsystemcog.org/site_media/logos/gfs4c.png">

## Archive GFS Forecast Grib Files
Some days are missing...

Roughly 10 days behind current date 

* https://www.ncdc.noaa.gov/data-access/model-data/model-datasets/global-forcast-system-gfs

In [1]:

import pygrib

# Random Library Imports
import os,glob,re,time

# Importing Datetime Libraries
from datetime import datetime, timedelta

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


# Numerical and Scientific Libraries
import numpy as np
from scipy.ndimage import gaussian_filter

# Matplotlib Plotting Libraries
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mticker

# Warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.gridspec as gridspec
print("done.")


done.


In [2]:
# Set the font 
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 18,
        }

In [3]:
cd ~/Downloads/

/Users/chowdahead/Downloads


In [4]:
import Vorticity_Color_Bar as vort_cmap
vort_cmapz = vort_cmap.my_cmap

## Use curl to download grib file from https://nomads.ncdc.noaa.gov/data/

In [27]:
year = "2019"
month = "11"
day = "10"
init_hour = "0000"
f_hour = "012"

! curl https://nomads.ncdc.noaa.gov/data/gfs4/{year}{month}/{year}{month}{day}/gfs_4_{year}{month}{day}_{init_hour}_{f_hour}.grb2 --output gfs_4_{year}{month}{day}_{init_hour}_{f_hour}.grb2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 96.4M  100 96.4M    0     0  5209k      0  0:00:18  0:00:18 --:--:-- 1382k


## Search the current working drive for files that start with gfs

In [28]:
ls gfs_*

gfs_4_20191110_0000_012.grb2  gfs_4_20191115_0000_027.grb2
gfs_4_20191115_0000_000.grb2  gfs_4_20191115_0600_003.grb2
gfs_4_20191115_0000_003.grb2  gfs_4_20191115_0600_030.grb2
gfs_4_20191115_0000_012.grb2


## Use ```glob``` to grab the names of all the gfs files

In [47]:
grib_list = glob.glob('gfs_4_20191115_0600_030.grb2')
grib_list = sorted(grib_list, key=lambda x: int(re.sub('\D', '', x)))
print(grib_list)

['gfs_4_20191115_0600_030.grb2']


## Grabbing the first grib file from our list will allow us to explore the data
* Each entry of the file is a different variable (specified at different heights for most)
* There are over 500 variables for a GFS grib file!!

In [48]:
#grib = "gfsanl_3_20190129_1200_000.grb2" # Set the file name of your input GRIB file
grbs = pygrib.open(grib_list[0])
grb = grbs.read()

### Since there's so many variables, lets just print out the first 10 to get an idea of what the file looks like

In [ ]:
grb[0:10]

### There is a built in search by name function, but the exact name has be specified

In [ ]:
grbs.select(name="MSLP")

In [ ]:
grbs.select(name="MSLP (Eta model reduction)")

In [ ]:
type(grbs.select(name="MSLP (Eta model reduction)"))

In [ ]:
grbs.select(name="Absolute vorticity",level=500)[0]

## Grab subsets of data based off variable name:

### Geopotential Heights:

In [ ]:
GeoHeight = [i for i in grbs.select(name="Geopotential Height")]
GeoHeight[0]

In [ ]:
Temp = [i for i in grbs.select(name="Temperature")]
Temp[0:5]

In [ ]:
AbVort = [i for i in grbs.select(name="Absolute vorticity")]
AbVort[0:5]


## Each entry in our new list is a ```pygrib.gribmessage```

* https://jswhit.github.io/pygrib/docs/pygrib.gribmessage-class.html

In [ ]:
type(GeoHeight[0])

## Analysis datetime ```analDate```

In [ ]:
GeoHeight[0].analDate

In [ ]:
GeoHeight[0].keys()

### Since it is easier to call the variables from the grib file by the entry index, the ```.messagenumber``` instance varibale will find that for us:

In [ ]:
GeoHeight[0].messagenumber,grbs[GeoHeight[0].messagenumber]

## Looking to find which index in the whole ```grib``` file is 500mb Geopotential Heights:
* Note: in the pygrib.gribmessage, the level is indicated with Pa, yet when calling the ```pygrib.gribmessage["level"]``` key, it has to be called with hPa. I have no idea why this is, and would have to dive a bit further into the docs...

Find the level of ```GeoHeight[0]```:

In [ ]:
GeoHeight[0]["level"]

In [ ]:
for i in GeoHeight:
    if i["level"] == 500: # !!Look for the 500 hPa not 50000 Pa!!
        mb500_lev_index = i.messagenumber
        print(i,f"\nGrib variable index: {i.messagenumber}")


### The data values, lats, and lons are in different entries of each variable speficied at a height

In [ ]:
GeoHeight[0].data() # comes out as arrays

# With this data now parsed, we can try plotting

### Clean up some of the variables we will plot by taking a lat/lon subset:

In [49]:
hgt_500 = grbs.select(name="Geopotential Height",level=500)[0].data()
hgt_500

(array([[5150.7676, 5150.7676, 5150.7676, ..., 5150.7676, 5150.7676,
         5150.7676],
        [5155.8076, 5155.9272, 5156.0474, ..., 5155.4272, 5155.5474,
         5155.6675],
        [5162.7476, 5163.0273, 5163.3076, ..., 5161.887 , 5162.1875,
         5162.4673],
        ...,
        [5104.5073, 5104.6875, 5104.847 , ..., 5103.9873, 5104.1675,
         5104.347 ],
        [5102.3076, 5102.367 , 5102.4673, ..., 5102.0474, 5102.1274,
         5102.2075],
        [5099.7476, 5099.7476, 5099.7476, ..., 5099.7476, 5099.7476,
         5099.7476]], dtype=float32),
 array([[ 90. ,  90. ,  90. , ...,  90. ,  90. ,  90. ],
        [ 89.5,  89.5,  89.5, ...,  89.5,  89.5,  89.5],
        [ 89. ,  89. ,  89. , ...,  89. ,  89. ,  89. ],
        ...,
        [-89. , -89. , -89. , ..., -89. , -89. , -89. ],
        [-89.5, -89.5, -89.5, ..., -89.5, -89.5, -89.5],
        [-90. , -90. , -90. , ..., -90. , -90. , -90. ]]),
 array([[  0. ,   0.5,   1. , ..., 358.5, 359. , 359.5],
        [  0. , 

In [50]:
mslp = grbs.select(name="MSLP (Eta model reduction)")[0] # [0] since MSLP is a list?
mslp_data, lats, lons = mslp.data(lat1=20,lat2=70,lon1=220,lon2=320)
mslp_data = gaussian_filter(mslp_data, 3)/100

hgt_500 = grbs.select(name="Geopotential Height",level=500)[0].data(lat1=20,lat2=70,lon1=220,lon2=320)
vort_500 = grbs.select(name="Absolute vorticity",level=500)[0].data(lat1=20,lat2=70,lon1=220,lon2=320)

### The date can be grabbed from the data as well:

In [52]:
print(f"Data Date: {mslp['dataDate']}")
print(f"Hour: {mslp['hour']}")
print(f"Minute: {mslp['minute']}")
print(f"Forecast Time: {mslp['forecastTime']}")
print(f"Data Time: {mslp['dataTime']}")

Date = mslp['dataDate']
InitDate = Date
Year = mslp['year']
Month = mslp['month']  
Day = mslp['day']

InitHour = int(str(mslp['dataTime'])[0])
print(InitHour)
ForecastStep = mslp['forecastTime']
ForecastDateTime = str(datetime(int(Year),int(Month),int(Day),int(InitHour)) + timedelta(hours=ForecastStep))
ForecastDateTime = ForecastDateTime[:-6]

ForecastDate = ForecastDateTime[:-3]
print(ForecastDate)
ForecastTime = ForecastDateTime[-2:]+"Z"
print(ForecastTime)
forecast_title_time = f'{ForecastTime} {ForecastDate}'
print(forecast_title_time)    


if InitHour < 10:
    InitHour = f"0{InitHour}"
if ForecastStep < 10:
    ForecastStep = f"00{ForecastStep}"
print(int(ForecastStep))      
if 10 < int(ForecastStep) < 100:
    ForecastStep = f"0{ForecastStep}"

      
title_date = f"{Year}-{Month}-{Day}" 
#if 24 < int(ForecastStep) < 48:
#    ForecastDay = Day + 1
#    forecast_hour = int(ForecastStep) - 24
#    if forecast_hour < 10:
#        forecast_hour = f"0{forecast_hour}"

#forecast_date = 
file_time = f"{InitDate}_{InitHour}00Z_F{ForecastStep}"

file_time, title_date,InitHour,forecast_title_time

Data Date: 20191115
Hour: 6
Minute: 0
Forecast Time: 30
Data Time: 600
6
2019-11-16
12Z
12Z 2019-11-16
30


('20191115_0600Z_F030', '2019-11-15', '06', '12Z 2019-11-16')

In [ ]:
print(datetime(int(Year),int(Month),int(Day),int(InitHour)) + timedelta(hours=ForecastStep))

In [20]:
# Add Map Features
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.218, 0.01, 0.589, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon0,lon1,lat0,lat1]
extent = [-180., -60, 20., 70.]

kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}

In [ ]:
# Create new figure
datacrs = ccrs.PlateCarree()
plotcrs = ccrs.NorthPolarStereo(central_longitude=-100.0)

fig = plt.figure(figsize=(17., 11.))

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)

ax.set_extent([-125, -70, 20, 55], ccrs.PlateCarree())

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='blue', linewidth=1,zorder=10)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1,zorder=10)

# Plot Title
ax.set_title('GFS ($0.5^o$) Forecast: MSLP, 500mb Heights and Abs Vorticity '+ "("+"$\mathregular{s^{-1}}$"+")", 
                 size=16, loc='left',fontdict=font)
    
ax.set_title(f"Init Hour: {InitHour}Z {title_date}\nForecast Hour: F{ForecastStep} - {forecast_title_time}",
                 size=16, loc='right',fontdict=font)

'''
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = True
gl.xlines = True
#gl.xlocator = mticker.FixedLocator([-125, -110, -95, -80, -70])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

gl.ylabel_style = {'size': 15, 'color': 'gray'}
gl.ylabel_style = {'color': 'red', 'weight': 'bold'}

gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.xlabel_style = {'color': 'red', 'weight': 'bold'}
'''

vort_levels = vort_cmap.vort_levels

cs = ax.contour(lons, lats, mslp_data,np.arange(970,1040,4),colors='k',linestyles='dashed',transform=datacrs)#cmap='nipy_spectral'
plt.clabel(cs, **kw_clabels)

cs2 = ax.contour(lons, lats, hgt_500[0],np.arange(5000,6000,75),transform=datacrs,colors="k",linewidth=1.5)

cs3 = ax.contourf(lons, lats, vort_500[0],vort_levels,transform=datacrs,cmap=vort_cmapz)
#cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]
#cbar = plt.colorbar(cs2, orientation='horizontal',cax=cbaxes)

plt.show()
outfile=f"/Users/chowdahead/Documents/GitHub/Python-Jupyter/Jupyter-Notebooks/Gribs-Examples/GFS_4_mslp_500_hgts_vort_{file_time}.png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)